In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
import os

In [4]:
os.chdir('C:\\Users\\steve\\OneDrive\\Data Science MSc\\MSc Project') # make sure we start right

##### Exploring the Data

In [5]:
df = pd.read_csv('THIS_IS_IT.csv')
df.tail()

,Image Data ID,Subject,Group,Sex,Age,Visit,Modality,Description,Type,Acq Date,Format,Downloaded
726,226399,3104,Control,M,72,1,MRI,Axial PD-T2 TSE,Original,2/14/2011,DCM,NaN
727,301554,3104,Control,M,73,5,MRI,Axial PD-T2 TSE,Original,4/20/2012,DCM,NaN
728,226366,3102,PD,M,64,1,MRI,Axial PD-T2 TSE,Original,11/23/2010,DCM,NaN
729,378210,3102,PD,M,66,7,MRI,Axial PD-T2 TSE FS,Original,02/06/2013,DCM,NaN
730,353457,3102,PD,M,65,21,MRI,Axial PD-T2 TSE,Original,11/21/2011,DCM,NaN


In [6]:
descriptions_array = df.Description.values
np.unique(descriptions_array, return_counts=True) 
# so most is Axial PD-T2 TSE and Axial PD-T2 TSE FS
# leaves us with either 292 or 356, try to find the difference

(array(['AXIAL PD-T2 TSE FS', 'Axial PD-T2 TSE', 'Axial PD-T2 TSE FS',
        'Axial PD-T2 TSE_AC PC line', 'Axial PD-T2 TSE_AC/PC line',
        'Axial PD-T2-FS TSE', 'T2'], dtype=object),
 array([ 33, 292, 356,   2,  24,  23,   1], dtype=int64))

In [7]:
# looks like a bunch of Axial PD-T2 TSE are mislabelled or slightly different, change these
dscrptn = df['Description'].replace({'Axial PD-T2-FS TSE':'Axial PD-T2 TSE FS', 'AXIAL PD-T2 TSE FS':'Axial PD-T2 TSE FS', 
                           'Axial PD-T2 TSE_AC PC line':'Axial PD-T2 TSE_AC/PC line'})

df['Description'] = dscrptn

In [8]:
np.unique(df.Description.values, return_counts=True) 
# looks about right. Now need to decide on TSE vs TSE FS
# well FS has more data, let's try to find something more substantial though

(array(['Axial PD-T2 TSE', 'Axial PD-T2 TSE FS',
        'Axial PD-T2 TSE_AC/PC line', 'T2'], dtype=object),
 array([292, 412,  26,   1], dtype=int64))

In [9]:
# find an example of a TSE and a TSE FS and try to find the difference

In [10]:
tse_fs_test_id = 377981
tse_nofs_test_id = 226399

##### Changing working directory to access the images

In [11]:
# have to dive into the file system to do that.. (on AWS have it all accessed on the bucketz)
import pydicom
from pydicom.data import get_testdata_files

In [12]:
os.getcwd()

'C:\\Users\\steve\\OneDrive\\Data Science MSc\\MSc Project'

In [33]:
os.chdir('C:\\Users\\steve\Downloads\PPMI_THIS_IS_IT')
os.getcwd()

'C:\\Users\\steve\\Downloads\\PPMI_THIS_IS_IT'

In [24]:
filename = get_testdata_files("\\3113\\Axial_PD-T2_TSE\\2012-07-18_13_26_33.0\\S169223.dcm")
print (filename)
# acknowledge when opening the files, the descriptors may be different, or make a crawler and look for image numbers
ds = pydicom.dcmread(filename)
plt.imshow(ds.pixel_array, cmap=plt.cm.bone) 

[]


AttributeError: 'list' object has no attribute 'read'